### 허깅페이스 모델 데이터세트 수집 코드

In [ ]:
from huggingface_hub import HfApi
import pandas as pd
from tqdm import tqdm
import ast
from datetime import datetime

now = datetime.now() # 현재 날짜와 시간 얻기

year = now.year # 연, 월, 일 각각의 변수에 저장
month = now.month
day = now.day

api = HfApi() # Hugging Face API 인스턴스 생성
models_generator = api.list_models(full=True) # 모델 리스트를 가져오기 (모델 메타 데이터 포함)

models_data = [] # 모델 메타 데이터를 저장할 리스트 초기화

for model in tqdm(models_generator, desc="Fetching model metadata"): # 모델 메타 데이터를 가져오는 동안 진행도 표시
    models_data.append(model)

df = pd.DataFrame(models_data) # 데이터프레임으로 변환
columns_to_drop = ['siblings', 'mask_token', 'card_data', 'widget_data', 'model_index', 'config', 'transformers_info', 'spaces', 'safetensors'] # 불필요한 컬럼 삭제
df = df.drop(columns=columns_to_drop)

df['arxiv'] = '' # 2. 새로운 컬럼 생성
df['dataset'] = ''
df['region'] = ''
df['license'] = ''

def process_tags(tags): # 3. tags 컬럼의 객체들을 각 컬럼에 입력하고 tags에서 삭제
    arxiv, dataset, region, license = '', '', '', ''
    new_tags = []
    for tag in tags:
        lower_tag = tag.lower()
        if lower_tag.startswith('arxiv:'):
            arxiv = tag
        elif lower_tag.startswith('dataset:'):
            dataset = tag
        elif lower_tag.startswith('region:'):
            region = tag
        elif lower_tag.startswith('license:'):
            license = tag
        else:  # 특정 객체를 삭제하지 않음
            new_tags.append(tag)
    return arxiv, dataset, region, license, new_tags

for index in tqdm(range(len(df)), desc="Processing tags"): # 진행도를 표시하면서 tags 컬럼 처리
    tags_data = df.at[index, 'tags']
    if isinstance(tags_data, str):
        tags_data = ast.literal_eval(tags_data)
    arxiv, dataset, region, license, new_tags = process_tags(tags_data)
    df.at[index, 'arxiv'] = arxiv
    df.at[index, 'dataset'] = dataset
    df.at[index, 'region'] = region
    df.at[index, 'license'] = license
    df.at[index, 'tags'] = new_tags

output_csv_file = f'./Model_{month:02d}-{day:02d}.csv' # 4. 수정된 데이터 프레임을 CSV 파일로 저장한다.
df.to_csv(output_csv_file, index=False)

c:\Users\kodonghwan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching model metadata: 1407537it [15:53, 1476.21it/s]
Processing tags: 100%|██████████| 1407537/1407537 [02:34<00:00, 9132.93it/s] 
